In [ ]:
# coding:utf-8
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

def gragh(result, epoch):    
    x = np.arange(0, epoch, 1)
    left = np.array(x)
    height = np.array(result)
    plt.plot(left, height)

In [ ]:
from deep_qnetwork_with_cnn import DeepQNetworkWithCNN
import random
import time
import copy
import numpy as np

ACTION = [0, 1, 2, 3]
NUM_HID1 = 3136
NUM_HID2 = 512
NUM_OUT  = 4
BATCH_SIZE = 32
EPOCH = 10000
GAMMA = 0.9
EPSIL = 1.0
MINEPSIL = 0.1
NUM_LIMIT_RECORD = 10**4
NUM_INIT_RECORD = 10**5
ProccerUnit = "CPU"
MODEL_NAME = "./DQN_model/breakout2.model"
RESULT_REWARD = []

STATE_LENGTH = 4  # 状態を構成するフレーム数
FRAME_WIDTH = 84  # リサイズ後のフレーム幅
FRAME_HEIGHT = 84  # リサイズ後のフレーム高さ
NO_OP_STEPS = 30

def agent_action(env, action):
    next_state, reward, terminal, info = env.step(action)
    return next_state, reward, terminal, info

def deep_q_learn(env, agent):
    agent.load_weight()
    agent.update_target_network()
    step = 0
    for episode in range(1, EPOCH):
        terminal = False
        sum_reward = 0
        next_obs = env.reset()  # 環境の初期化、初期画面を返す
        for i in range(random.randint(1, NO_OP_STEPS)):  # ランダムなフレーム数分「何もしない」行動で遷移させる
            now_obs = next_obs
            next_obs, _, _, _ = env.step((i%4))  # 「何もしない」行動を取り、次の画面を返す
        now_state = agent.get_initial_state(now_obs, next_obs)  # 初期状態を作る
        last = 5
        learn_flg = 0
        epsil = agent.EPSIL
        if episode % 4 == 0:
            agent.EPSIL = 0.05
            learn_flg = 1
        while(not terminal):
            # 状態更新
            now_obs = copy.deepcopy(next_obs)
            # 方策に従って行動を選択
            action, _, action_list = agent.policy_egreedy(now_state, agent.Q_network, step)
            # 行動による次状態を観測
            next_obs, reward, term, info = agent_action(env, ACTION[action])
            # ゴールの設定
            terminal, reward = (True, reward) if term else (False, reward)
            
            # if last != info["ale.lives"]:
            #     reward = -1.0
            
            preprocess_obs = agent.pre_proccesing(now_obs, next_obs)
            next_state = np.append(now_state[1:, :, :], np.array([preprocess_obs]), axis=0)
            sum_reward += reward
            agent.stock_record(now_state, action, next_state, reward, terminal, action_list)

            env.render()
            if terminal:
                break
                
            # 入力状態更新
            now_state = copy.deepcopy(next_state)
            last = info["ale.lives"]
            
            # Experience_Replayによるバッチ学習
            if len(agent.record) > NUM_LIMIT_RECORD:
                agent.init_record()
                if learn_flg:
                    agent.experience_replay_ver2()
            if agent.EPSIL > MINEPSIL: agent.EPSIL -= EPSIL / EPOCH / 10
            if step % 10**4 == 0:
                agent.update_target_network()               
            step += 1
            if step % 500 == 0: print("step:" + str(step))
        
        print(episode, sum_reward, agent.EPSIL, step)
        agent.EPSIL = epsil
        RESULT_REWARD.append(sum_reward)
        agent.save_weight()
    gragh(RESULT_REWARD, len(RESULT_REWARD))

if __name__=="__main__":
    import gym
    env = gym.make("Breakout-v0")
    agent = DeepQNetworkWithCNN(NUM_HID1, NUM_HID2, NUM_OUT,ProccerUnit, EPSIL, GAMMA, BATCH_SIZE, MODEL_NAME)
    deep_q_learn(env, agent)

[2017-06-11 22:32:55,056] Making new env: Breakout-v0


[0, 1, 2, 3]
Nothing
1 2.0 0.9970400000000135 296
step:500
2 2.0 0.9974000000000118 556
3 0.0 0.9986100000000063 695
4 2.0 0.05 974
step:1000
5 1.0 0.9979900000000091 1175
6 0.0 0.9983500000000075 1340
step:1500
7 2.0 0.9972200000000127 1618
8 2.0 0.05 1914
step:2000
9 1.0 0.9979800000000092 2116
10 0.0 0.9985800000000065 2258
11 1.0 0.99781000000001 2477
step:2500
12 1.0 0.05 2714
13 1.0 0.9977700000000101 2937
step:3000
14 2.0 0.9972800000000124 3209
15 0.0 0.998250000000008 3384
step:3500
16 3.0 0.05 3677
17 0.0 0.9984400000000071 3833
step:4000
18 1.0 0.9979300000000094 4040
19 3.0 0.996700000000015 4370
step:4500
20 1.0 0.05 4571
21 0.0 0.9983800000000074 4733
22 2.0 0.9973500000000121 4998
step:5000
23 4.0 0.9963700000000165 5361
step:5500
24 1.0 0.05 5590
25 2.0 0.997580000000011 5832
step:6000
26 3.0 0.9969600000000138 6136
27 2.0 0.9973400000000121 6402
step:6500
28 0.0 0.05 6576
29 0.0 0.9981900000000082 6757
30 0.0 0.9982900000000078 6928
step:7000
31 3.0 0.9963900000000164 